# Quick test outside of Test macro

In [5]:
using GalacticDynamics
using Test
using PythonCall
using DifferentialEquations


pyimport("sys")."path".append("")
pyimport("sys")."path".append("python")
accelerations_py = pyimport("accelerations")
au = pyimport("astropy.units")
gd = pyimport("gala.dynamics")
gp = pyimport("gala.potential")
gu = pyimport("gala.units")
gi = pyimport("gala.integrate")

Python: <module 'gala.integrate' from '/home/mmestre/.conda/envs/cosmos/lib/python3.11/site-packages/gala/integrate/__init__.py'>

In [38]:

t₁, t₂ = 0.0, 2.0
t_range = (t₁, t₂)
Δt = 0.01
x₀ = 10*[1,0,0]
v₀ = 100*[0,1,0]
m = 10^12*𝕦.m  # Msun
a = 20*𝕦.l
pot = NFW(m, a)
c = concentration(pot)
f(x) = log(1+x)-x/(1+x)
m_g = m/f(c)
usys = gu.UnitSystem(au.kpc, au.Gyr, au.Msun, au.radian, au.km/au.s, au.km/au.Gyr^2)
pot_Gala = gp.NFWPotential(Py(adimensional(m_g))*au.Msun, Py(adimensional(a))*au.kpc, units=usys)
w₀ = gd.PhaseSpacePosition(pos=Py(x₀)*au.kpc, vel=Py(v₀)*au.km/au.s)

Python: <PhaseSpacePosition cartesian, dim=3, shape=()>

In [47]:
sol_gala = pot_Gala.integrate_orbit(w₀, dt=Δt*au.Gyr, t1=t₁, t2=(t₂+Δt)*au.Gyr,
                                    Integrator=gi.DOPRI853Integrator,
                                    Integrator_kwargs=Dict("rtol"=>5.0e-9, "atol"=>0.5e-9))
sol = evolve(pot, x₀, v₀, t_range; options=ntSolverOptions(; saveat=Δt))
@show 𝕤.ode
@show sol_gala.t
@show sol.t
@show sol_gala.x
@show sol.x[1,:]
orb_gala_x = pyconvert(Vector{Float64}, sol_gala.x)
orb_x = sol.x[1,:]
sol_dif = abs.(orb_gala_x-orb_x)
@show findmax(sol_dif)
@show maximum(sol_dif)


𝕤.ode = Vern8(; stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false), lazy = true,)
sol_gala.t = <py Quantity [0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09,
           0.1 , 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19,
           0.2 , 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29,
           0.3 , 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39,
           0.4 , 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49,
           0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59,
           0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69,
           0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79,
           0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89,
           0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
           1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09,
           1.1 , 1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19,
         

2.2603498981128323

In [51]:
sol = evolve(pot, x₀, v₀, t_range, Vern7(); options=ntSolverOptions(; saveat=Δt))
@show sol.x;

sol.x = [10.0 10.89569654712676 11.588000256102779 12.083075367940987 12.385141192395608 12.496698661487171 12.41865776477321 12.150382676478769 11.68966029373267 11.03258821127346 10.173367498333134; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]


In [35]:
sol = evolve(pot, x₀, v₀, t_range, DP8(); options=ntSolverOptions(; saveat=Δt))
@show sol.x;

sol.x = [10.0 10.895696539341532 11.588000260948066 12.083075365380834 12.385141185683452 12.496698660861693 12.418657761370094 12.15038266552423 11.68966025897951 11.032588274154195 10.173367497878917; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]


In [27]:

norm(2,3)

UndefVarError: UndefVarError: `norm` not defined